# Import libraries

In [54]:
import requests
import configparser
import pandas as pd
import math

# Load credentials

In [22]:
# Define credentials
config = configparser.ConfigParser()
config.read('config.ini')
# Get Hevy API key
hevy_api_key = config['Hevy']['API_KEY']


# Get workouts count

In [51]:
def get_workout_count():
    url = f'https://api.hevyapp.com/v1/workouts/count'
    headers = {
        'accept': 'application/json',
        'api-key': hevy_api_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        print('Workout count:', data['workout_count'])
        return int(data['workout_count'])
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [52]:
workout_count = get_workout_count()

Workout count: 119


# Get workouts data

In [94]:
def get_workouts(page = int, pageSize = int):
    ''' 
    Fetches workout data from the Hevy API and organizes it into DataFrames.

    Parameters:
    - page (int): The page number to start fetching workouts from (1-indexed).
                   Must be an integer greater than  1.
    - pageSize (int): The number of workouts to fetch per page.
                      Must be an integer in the range [1, 10] and greater than 'page'.

    Returns:
    - df_workouts (pd.DataFrame): A DataFrame containing distinct workout details.
    - df_exercises (pd.DataFrame): A DataFrame containing details of exercises performed during the workouts.
    - df_sets (pd.DataFrame): A DataFrame containing details of each set performed within the exercises
    '''

    if not (1 <= page):
        raise ValueError("Page must be greater than 1.")
    #if not (1 <= pageSize <= 10):
    #    raise ValueError("PageSize must be in the range [1, 10].")

    url = f'https://api.hevyapp.com/v1/workouts?page={page}&pageSize={pageSize}&since=1970-01-01T00%3A00%3A00Z'
    headers = {
        'accept': 'application/json',
        'api-key': hevy_api_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        workouts = data['workouts']

        # Initialize lists to collect data
        workout_list = []
        exercise_list = []
        set_list = []

        # Loop through workouts
        for workout in workouts:
            workout_id = workout['id']
            workout_title = workout['title']
            workout_description = workout['description']
            start_time = workout['start_time']
            end_time = workout['end_time']

            # Add workout to the list
            workout_list.append({
                'workout_id': workout_id,
                'title': workout_title,
                'description': workout_description,
                'start_time': start_time,
                'end_time': end_time
            })

            # Loop through exercises
            for exercise in workout['exercises']:
                exercise_index = exercise['index']
                exercise_title = exercise['title']
                exercise_notes = exercise['notes']
                exercise_template_id = exercise['exercise_template_id']
                superset_id = exercise['superset_id']

                # Add exercise to the list
                exercise_list.append({
                    'workout_id': workout_id,
                    'exercise_index': exercise_index,
                    'title': exercise_title,
                    'notes': exercise_notes,
                    'exercise_template_id': exercise_template_id,
                    'superset_id': superset_id
                })

                # Loop through sets
                for set_ in exercise['sets']:
                    set_index = set_['index']
                    set_type = set_['set_type']
                    weight_kg = set_['weight_kg']
                    reps = set_['reps']
                    distance_meters = set_['distance_meters']
                    duration_seconds = set_['duration_seconds']
                    rpe = set_['rpe']

                    # Add set to the list
                    set_list.append({
                        'workout_id': workout_id,
                        'exercise_index': exercise_index,
                        'set_index': set_index,
                        'set_type': set_type,
                        'weight_kg': weight_kg,
                        'reps': reps,
                        'distance_meters': distance_meters,
                        'duration_seconds': duration_seconds,
                        'rpe': rpe
                    })

        # Create DataFrames
        df_workouts = pd.DataFrame(workout_list)
        df_exercises = pd.DataFrame(exercise_list)
        df_sets = pd.DataFrame(set_list)
        return df_workouts, df_exercises, df_sets
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [114]:
df_workouts = pd.DataFrame()
df_exercises = pd.DataFrame()
df_sets = pd.DataFrame()

for page in range(math.ceil(workout_count/10)):
    page = page + 1 
    df_workouts_temp, df_exercises_temp, df_sets_temp = get_workouts(page,10)

    df_workouts = pd.concat([df_workouts, df_workouts_temp], ignore_index=True)
    df_exercises = pd.concat([df_exercises, df_exercises_temp], ignore_index=True)
    df_sets = pd.concat([df_sets, df_sets_temp], ignore_index=True)
    

C:\Users\MikolajPawlak\AppData\Local\Temp\ipykernel_23144\2460277354.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_exercises = pd.concat([df_exercises, df_exercises_temp], ignore_index=True, sort=False)
C:\Users\MikolajPawlak\AppData\Local\Temp\ipykernel_23144\2460277354.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_sets = pd.concat([df_sets, df_sets_temp], ignore_index=True, sort=False)


## Inspect results

In [121]:
print('Workouts len: ', len(df_workouts))
df_workouts.iloc[7:10]

Workouts len:  119


,workout_id,title,description,start_time,end_time
7,97e43acd-04b3-49c6-b8cc-4a083780e055,"Wieczorny bieg, 6:02min/km",,2024-05-21T19:34:37+00:00,2024-05-21T20:01:24+00:00
8,3dc499ea-e4c5-4c4b-b1d4-4e4baea6fe7a,Jest trening jest kebab,,2024-05-19T11:23:04+00:00,2024-05-19T15:21:02+00:00
9,75c7eba1-2982-45fe-b873-4cd7bd6989e4,Wieczorny trucht,,2024-05-16T21:28:43+00:00,2024-05-16T21:57:51+00:00


In [117]:
print('Excercises len: ', len(df_exercises))
df_exercises.iloc[10:15]

Excercises len:  693


,workout_id,exercise_index,title,notes,exercise_template_id,superset_id
10,631654b9-da99-4512-8c41-f93e2b4fff6d,1,Pull Up,,1B2B1E7C,NaN
11,631654b9-da99-4512-8c41-f93e2b4fff6d,2,Iso-Lateral Row (Machine),,AA1EB7D8,NaN
12,631654b9-da99-4512-8c41-f93e2b4fff6d,3,Incline Bench Press (Smith Machine),,3A6FA3D1,NaN
13,631654b9-da99-4512-8c41-f93e2b4fff6d,4,Pullover (Machine),,B123DD01,NaN
14,631654b9-da99-4512-8c41-f93e2b4fff6d,5,Front Raise (Dumbbell),,8293E554,NaN


In [104]:
print('Sets len: ', len(df_sets))
df_sets.head()

Sets len:  2764


,workout_id,exercise_index,set_index,set_type,weight_kg,reps,distance_meters,duration_seconds,rpe
0,5fec8b04-aa57-44ca-bfab-8b64e9f0459e,0,0,warmup,30.0,20.0,NaN,NaN,None
1,5fec8b04-aa57-44ca-bfab-8b64e9f0459e,0,1,normal,70.0,6.0,NaN,NaN,None
2,5fec8b04-aa57-44ca-bfab-8b64e9f0459e,0,2,normal,65.0,8.0,NaN,NaN,None
3,5fec8b04-aa57-44ca-bfab-8b64e9f0459e,0,3,failure,65.0,8.0,NaN,NaN,None
4,5fec8b04-aa57-44ca-bfab-8b64e9f0459e,0,4,normal,60.0,9.0,NaN,NaN,None


# Save results

In [107]:
# Merge all dataframes
df_full = pd.merge(df_workouts, df_exercises, how = 'left', on= 'workout_id')
df_full = pd.merge(df_full, df_sets, how = 'left', on= ['workout_id', 'exercise_index'])

In [122]:
# Adjust column names
df_full.rename(columns={'title_x': 'workout_title', 'title_y': 'excercise_title'}, inplace= True)
# Save data to a csv file
df_full.to_csv('hevy_workouts.csv')